In [1]:
import quartz
from QGNN import QGNN, train_supervised
import torch

Using backend: pytorch[22:20:13] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/lzk/anaconda3/envs/quantum/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory



In [2]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')
parser = quartz.PyQASMParser(context=quartz_context)
my_dag = parser.load_qasm(filename="../circuit/nam-circuits/qasm_files/tof_3_before.qasm")
my_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)
my_graph_dgl = my_graph.to_dgl_graph()
appliable_xfer_matrix = my_graph.get_available_xfers_matrix(context=quartz_context)
my_graph_dgl.ndata['label'] = torch.tensor(appliable_xfer_matrix,dtype=torch.float)

In [3]:
num_data = my_graph.num_nodes
val_rate = 0.15
test_rate = 0.15
val_num = int(num_data * val_rate)
test_num = int(num_data * test_rate)
train_num = num_data - val_num - test_num
train_mask = [1] * train_num + [0] * val_num + [0] * test_num
val_mask = [0] * train_num + [1] * val_num + [0] * test_num
test_mask = [0] * train_num + [0] * val_num + [1] * test_num

my_graph_dgl.ndata['train_mask'] = torch.tensor(train_mask,dtype=torch.bool) 
my_graph_dgl.ndata['val_mask'] = torch.tensor(val_mask,dtype=torch.bool) 
my_graph_dgl.ndata['test_mask'] = torch.tensor(test_mask,dtype=torch.bool) 
model = QGNN(26, 16, quartz_context.num_xfers, 16)
train_supervised(my_graph_dgl, model)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x8 and 29x16)